Imports

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
import pickle
import sys

ImportError: Could not find 'cudart64_100.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Download and install CUDA 10.0 from this URL: https://developer.nvidia.com/cuda-90-download-archive

In [2]:
# Function to show various error measurement
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    mse = np.mean((forecast - actual)**2)  # MSE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    #corr = np.corrcoef(forecast, actual)[0,1]   # corr
    #mins = np.amin(np.hstack([forecast[:,None], 
    #                          actual[:,None]]), axis=1)
    #maxs = np.amax(np.hstack([forecast[:,None], 
    #                          actual[:,None]]), axis=1)
    #minmax = 1 - np.mean(mins/maxs)             # minmax
    #acf1 = acf(fc-test)[1]                      # ACF1
    #return({'mape':mape, 'me':me, 'mae': mae, 
    #        'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
    #        'corr':corr, 'minmax':minmax})
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'mse':mse, 'rmse':rmse})

In [3]:
import os
os.chdir('d:/Users/mokky/Documents/GitHub/nus-iss/PRS-PM-ISY5002-GROUP5/SystemCode')
print(os.getcwd())

d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5\SystemCode


In [4]:
file_dir = './Data Exploration/data/uc3/'
X_train_file_path = 'all_hosp_new_train_X_uc3.pkl'
y_train_file_path = 'all_hosp_data_uc3_y_train_uc3.pkl'
X_test_file_path = 'all_hosp_new_test_X_uc3.pkl'
y_test_file_path = 'all_hosp_data_uc3_y_test_uc3.pkl'

In [5]:
df_X_train = pd.read_pickle(file_dir + X_train_file_path)
df_y_train = pd.read_pickle(file_dir + y_train_file_path)
df_X_test = pd.read_pickle(file_dir + X_test_file_path)
df_y_test = pd.read_pickle(file_dir + y_test_file_path)

Neural Networks ( Sequential )

In [11]:
NN_model = Sequential()

In [12]:
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = df_X_train.shape[1], activation='relu'))

In [13]:
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

In [14]:
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

In [15]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               618240    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 783,105
Trainable params: 783,105
Non-trainable params: 0
_________________________________________________________________


In [16]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(df_X_train, df_y_train, epochs=100, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

# save model to file
NN_model.save(file_dir + "uc3_NN_model.h5")

Train on 37510 samples, validate on 9378 samples
Epoch 1/100
 2944/37510 [=>............................] - ETA: 23:08:48 - loss: 5121.6778 - mean_absolute_error: 5121.6782

In [ ]:
predictions = NN_model.predict(df_X_test)

In [ ]:
sns.regplot(df_y_test,predictions)

In [ ]:
# MSE
print('Mean squared error: %.2f'
      % mean_squared_error(df_y_test,predictions, squared=True))

# RMSE
print('Root Mean squared error: %.2f'
      % mean_squared_error(df_y_test,predictions, squared=False))

# MAE
print('Mean absolute error: %.2f'
      % mean_absolute_error(df_y_test,predictions))

In [ ]:
forecast_accuracy(predictions, df_y_test)